In [1]:
import pickle as pkl

from torch.optim import AdamW
import torch.nn as nn
import torch

from dataset.dataset import SaladsDataset
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from tensorboardX import SummaryWriter
from src.ddpm.ddmp_multinomial import Diffusion
from src.ddpm.modules import UNet

In [2]:
num_epochs = 1000
batch_size = 10
device = "cuda"

In [3]:
with open("data/pickles/50_salads_one_hot.pkl", "rb") as f:
    dataset = pkl.load(f)

train_dataset, test_dataset = train_test_split(dataset, train_size=0.8, shuffle=True, random_state=17)
train_salads = SaladsDataset(train_dataset)
test_salads = SaladsDataset(test_dataset)

train_loader = DataLoader(train_salads, batch_size=batch_size,
                          collate_fn=lambda batch: pad_sequence(batch, batch_first=True, padding_value=-1))
test_loader = DataLoader(test_salads, batch_size=batch_size,
                         collate_fn=lambda batch: pad_sequence(batch, batch_first=True, padding_value=-1))

In [4]:
pg_batch = next(iter(train_loader)).to(device)
pg_batch.shape

torch.Size([10, 6584, 19])

In [5]:
diffuser = Diffusion()

t = diffuser.sample_timesteps(20)
batch_noise_steps, noise = diffuser.noise_data(pg_batch, t)
# dims: t_noise, b, t_data, c 
batch_noise_steps.shape

torch.Size([20, 10, 6584, 19])

In [6]:
from torch.nn import TransformerEncoderLayer, LSTM

# denoiser = Transformer(d_model=19, nhead=19, num_encoder_layers=2, batch_first=True).to(device)
denoiser = LSTM(input_size=19, hidden_size=512, batch_first=True).to(device)
out, (hn, cn) = denoiser(batch_noise_steps[0].to(device).float())

In [7]:
batch_noise_steps[0].shape

torch.Size([10, 6584, 19])

In [8]:
pg_batch.shape

torch.Size([10, 6584, 19])

In [11]:
trans = Transformer(d_model=512, nhead=8, num_encoder_layers=6, num_decoder_layers=6)
trans(out, pg_batch)

C:\Users\user\Anaconda3\envs\everything\Lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


RuntimeError: the feature number of src and tgt must be equal to d_model